In [2]:
import codecs

In [3]:
lines=codecs.open('1.txt',encoding='utf8')
arr1=[]  #  garbage_text
arr2=[]
for line in lines.readlines()[0:30000]:
    arr=line.strip().split('\t')[1:3]
    if arr[0]=='1':
        arr1.append(arr)
    else :
        arr2.append(arr)

In [4]:
arr1[0:5]

[['1',
  '南口阿玛施新春第一批限量春装到店啦\ue310   \ue310   \ue310   春暖花开淑女裙、冰蓝色公主衫\ue006   气质粉小西装、冰丝女王长半裙、\ue319   皇'],
 ['1', '感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】'],
 ['1', '一次价值xxx元王牌项目；可充值xxx元店内项目卡一张；可以参与V动好生活百分百抽奖机会一次！预约电话：xxxxxxxxxxx'],
 ['1', '(长期诚信在本市作各类资格职称（以及印 /章、牌、 ……等。祥：x x x x x x x x x x x 李伟%'],
 ['1', '《依林美容》三．八．女人节倾情大放送活动开始啦！！！！超值套餐等你拿，活动时间x月x日一x月xx日，     详情进店咨询。美丽热线x']]

In [5]:
totals={'y':len(arr1),'n':len(arr2)}
totals

{'n': 26986, 'y': 3014}

# 先读垃圾短信  找出其中最常出现的词汇，存到黑名单

In [6]:
import jieba.analyse
import codecs
import re   #正则
from collections import defaultdict
str1=''
for arr in arr1:   # arr1 垃圾短信
    str1+=re.sub(r'x+','',arr[1])  #将数字替换成 空  
content=str1
#加入停用词
jieba.analyse.set_stop_words('ting.txt')

tags=jieba.analyse.extract_tags(content,topK=200000,withWeight=True)
#topK  为返回几个 TF/IDF 权重大的几个关键词    默认为20
# withWeight 是否一并返回关键词于权重  默认 False

print(len(tags))
nums_list_y=defaultdict(float)

#将 tags中前200个词当成 拒绝的词
with codecs.open('black1.txt','a','utf8')as fp:
    for i,j in tags[:200]:
        fp.write(i+'\r\n')
        
for i,j in tags:
    nums_list_y[i]=j

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\吴彦祖\AppData\Local\Temp\jieba.cache
Loading model cost 0.798 seconds.
Prefix dict has been built succesfully.


14131


# 读取非垃圾短信

In [7]:
str1=''
for arr in arr2:
    str1+=re.sub(r'x+','',arr[1])
content=str1

jieba.analyse.set_stop_words('ting.txt')
tags=jieba.analyse.extract_tags(content,topK=200000,withWeight=True)

print(len(tags))
nums_list_n=defaultdict(float)
       
for i,j in tags:
    nums_list_n[i]=j

44977


In [8]:
nums_list_y

defaultdict(float,
            {'您好': 0.07381718173533189,
             '活动': 0.06382336889084934,
             '优惠': 0.053255896616915585,
             '地址': 0.04350363985436574,
             '全场': 0.04129157113093391,
             '祝您': 0.03173867716804985,
             '电话': 0.03002286249160352,
             '顾客': 0.02997808823497926,
             '光临': 0.029183835879409727,
             '咨询': 0.02635973455149472,
             '免费': 0.026297561043436456,
             '致电': 0.025616005392114647,
             '报名': 0.024398289836636278,
             '你好': 0.024076611829616904,
             '尊敬': 0.023672743513520812,
             '本店': 0.023642209120439198,
             '欢迎您': 0.023504789090941513,
             '赠送': 0.023454741720050493,
             '女人': 0.022754061621526177,
             '专柜': 0.02244957188090725,
             '元宵节': 0.02212368868592398,
             '礼品': 0.022011674383611773,
             '推出': 0.021862391453745016,
             '新年好': 0.021446075441383317,
    

In [9]:
#使用多项式计算概率
def laplace(char,num_list,totals,alpha=1):
    num_list.setdefault(char,0.0)
    count=num_list[char]*totals
    smooth=(alpha+count)/(totals+len(num_list)*alpha)
    return smooth

# 当新词出现时  1/3000  , 1/27000 更喜欢将其归于  垃圾短信

In [10]:
laplace('您好',nums_list_y,totals['y'])  #您好在垃圾短信里面概率

0.013034994794417634

In [11]:
laplace('您好',nums_list_n,totals['n'])  #您好在非垃圾短信里面概率

1.3895836807292535e-05

In [12]:
import math
def getLog(char,num_list,total,alpha=0.001):
    smooth=laplace(char,num_list,total,alpha)
    return math.log(smooth)

In [13]:
getLog('您好',nums_list_y,totals['y'])  #您好在垃圾短信里面概率

-2.6108367622706385

In [14]:
getLog('您好',nums_list_n,totals['n'])  #您好在非垃圾短信里面概率

-17.112494099618054

In [15]:
fr=codecs.open('black.txt',encoding='utf8')
black=[]

for line in fr.readlines():
    black.append(line.strip())
print(black)

['您好', '活动', '优惠', '地址', '全场', '祝您', '电话', '顾客', '光临', '咨询', '免费', '致电', '报名', '你好', '尊敬', '本店', '欢迎您', '赠送', '女人', '专柜', '元宵节', '礼品', '推出', '新年好', '特价', '回馈', '享受', '会员', '三八', '三八节', '进店', '热线', '感谢您', '选购', '三八妇女节', '快乐', '公司', '来电', '专业', '满元', '详询', '老客户', '即可', '新老', '优惠活动', '亲爱', '原价', '客户', '好消息', '特惠', '产品', '.%', '元宵', '春装', '送元', '抵押', '期待', '美丽', '惊喜', '现金', '时间', '买一送一', '服务', '三月', '购买', '装修', '新款', '办理', '装饰', '品牌', '前来', '当天', '红包', '羊年', '新年', '平米', '销售', '妇女节', '联系电话', '抢购', '有限', '减元', '家长', '咨询电话', '精美', '奶粉', '开课', '家装', '美容', '有限公司', '朋友', '预约', '超值', '贷款', '需元', '新品', '年月日', '感谢', '手机', '新春', '短信', '顾问', '护理', '话费', '教育', '家居', '提前', '积分', '折后', '感恩', '价格', '参加', '项目', '体验', '促销', '户型', '提供', '价值', '消费', '节日快乐', '满送', '名额有限', '谢谢', '辅导', '课程', '为您服务', '亲们', '宽带', '详情', '系列', '期间', '春季', '先得', '大型', '开年', '现场', '一年', '满减', '老师', '空调', '信用贷款', '广场', '养生', '详情请', '支持', '化妆品', '火热', 'VIP', '低至', '团购', '套餐', '喜迎', '预存', '即日起', '面膜', '酒店', '限量', '最低', '二楼', '充值', 'com'

In [23]:
def gaosi(x,a=1,b=0,c=50):
    return a*math.e**(-(x-b)**2/(2*c**2))

def computLogProb(strs,nums_list_y,nums_list_n,totals,blackWeight=1.2):
    log_n=0
    log_y=0
    content=re.sub(r'x+','',strs)
    jieba.analyse.set_stop_words('ting.txt')
    tags=jieba.analyse.extract_tags(content,topK=50,withWeight=True)
    
    for i,j in tags:
        if i in black:
            log_y+= ( getLog(i,nums_list_y,totals['y'])+math.log(blackWeight))
            #如果在黑名单中的词汇可以确定是一条垃圾短信，所以增加权重
            #更好的算法对黑名单中的单词按出现的频次 加权
            log_n+=getLog(i,nums_list_n,totals['n'])
        else:
            log_n+=getLog(i,nums_list_n,totals['n'])
            log_y+=getLog(i,nums_list_y,totals['y'])
    # p(c)   这项要考虑       totals['y']/(totals['n']+totals['y'])
    #先验概率   垃圾短信
    log_y+=math.log(totals['y']/(totals['n']+totals['y']))
    log_n+=math.log(totals['n']/(totals['n']+totals['y']))
    return {'n':log_n,'y':log_y}
teststr='欢迎致电绍兴赛荣电器服务中心！本店长期提供松下电器折扣商品（如松下电视机，空调，冰箱，微波炉，电饭煲等）。最低x折起。希望得到您一如既'
print(computLogProb(teststr,nums_list_y,nums_list_n,totals))

{'n': -182.33230211206737, 'y': -156.15167634291205}


In [26]:
result=computLogProb(teststr,nums_list_y,nums_list_n,totals)
if result['y'] > result['n']:
    print(teststr,'是垃圾短信')

欢迎致电绍兴赛荣电器服务中心！本店长期提供松下电器折扣商品（如松下电视机，空调，冰箱，微波炉，电饭煲等）。最低x折起。希望得到您一如既 是垃圾短信


In [18]:
#读取后一万条数据
import codecs
fr=codecs.open('1.txt',encoding='utf8')
label=[]
train=[]
for line in fr.readlines()[-10000:-1]:
    arr=line.strip().split('\t')
    label.append(arr[1])
    train.append(arr[2])
    
print(label[0])
print(train[0])

0
中国旅游网站艺龙接到现有股东腾讯的私有化提议


In [27]:
def test(strs,label,totals,num_list_y,num_list_n):
    count=0
    err=0
    result={}
    for str1 in strs:
        a='0'
        log_prob=computLogProb(str1,nums_list_y,nums_list_n,totals)
        if log_prob['y'] > log_prob['n']:
            a='1'
        if label[count]!=a:
            err+=1
        result.setdefault(label[count],{})
        result[label[count]].setdefault(a,0)
        result[label[count]][a]+=1
        count+=1
    print('错误率：',float(err/count))
    return (result,count)

In [28]:
mertrics,count=test(train,label,totals,nums_list_y,nums_list_n)
print(mertrics)
print(count)

错误率： 0.06500650065006501
{'0': {'0': 8370, '1': 613}, '1': {'1': 979, '0': 37}}
9999


In [30]:
#计算模型准确率
def showAccuracy(metrics):
    preRight=0  # TP + TN
    total=0
    for category,preDict in metrics.items():
        preRight+=metrics[category][category]
        for ca,result in preDict.items():
            total+=result
    return preRight/total

show_right=showAccuracy(mertrics)
print(show_right)

0.934993499349935


In [37]:
#计算模型精确率  对1的精确率   9/(9+6)    对0的精确率  21/(21+6)
def showPrecision( metrics):
    n=len(metrics)
    precision={}
    for i in range(n):
        ySum=sum(mertrics[str(row)][str(i)] for row in range(n) )
        p=metrics[str(i)][str(i)]/ySum
        precision[str(i)]=p
    return precision

precision_test=showPrecision(mertrics)
print(precision_test)

{'0': 0.9955989056738432, '1': 0.6149497487437185}


In [38]:
#计算召回率   对1的召回率  9/(9+4)   对0:  21/(21+6)
def showRecall(metrics):
    n=len(metrics)
    recall={}
    for i in range(n):
        xSum=sum(value for category,value in metrics[str(i)].items())
        recall[str(i)]=metrics[str(i)][str(i)]/xSum
    return recall

recall_test=showRecall(mertrics)
print(recall_test)

{'0': 0.9317599910942892, '1': 0.9635826771653543}


In [39]:
def kappa(categories,total):
    totalForP0=0
    for key,value in categories.items():
        totalForP0+=value[key]
    p0=totalForP0/total
    print('准确数：',totalForP0,'样本总数：',total)
    
    totalForPe=0
    for key,value in categories.items():
        xSum=0
        ySum=0
        for (k,v) in value.items():
            ySum+=categories[k][key]
            xSum+=v
        print(xSum,'--------',ySum)
        totalForPe+=xSum*ySum
    pe=totalForPe/(total*total)
    k=(p0-pe)/(1-pe)
    return k

In [40]:
kappa(mertrics,count)

准确数： 9349 样本总数： 9999
8983 -------- 8407
1016 -------- 1592


0.715470513493125

In [41]:
def getGender(logProb):
    if logProb['y']>logProb['n']:
        print('这是条垃圾短信')
        return 1
    else:
        print('这是条正常短信')
        return 0
    
str1='是汉奸还是被强奸自己对号入座'
result=computLogProb(str1,nums_list_y,nums_list_n,totals)
getGender(result)

这是条正常短信


0